In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import json
import re
from collections import defaultdict
from processing_pdf import processing_pdf

Define the input and output path.
> &#x26a0;&#xfe0f; **The last line will delete all files in the output folder.**


In [3]:
# path to the folder containing PDF files (1512, removed researchpaper1.pdf)
dataset_path = "F:\Datasets/"
# output folder and file name
output_path = "dataset2"
processing_pdf.clear_processed_folder(output_path)

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Kotorin\AppData\Local\Temp\ipykernel_19884\826408512.py:2: SyntaxWarning: invalid escape sequence '\D'
  dataset_path = "F:\Datasets/"


Remove duplicated files, only keep the papers (941) with newest version

In [ ]:
# Function to extract the base filename without the version suffix
def get_base_filename(filename):
    match = re.match(r'^(.*?)v\d\.pdf+$', filename)
    if match:
        return match.group(1)
    else:
        return filename

# Function to find the newest version of each file
def find_newest_versions(folder):
    files_by_base_name = defaultdict(list)

    # Group files by base filename
    for filename in os.listdir(folder):
        base_name = get_base_filename(filename)
        files_by_base_name[base_name].append(filename)

    # Find the newest version of each file
    newest_versions = []
    for base_name, filenames in files_by_base_name.items():
        newest_version = max(filenames, key=lambda x: int(re.search(r'v(\d+)\.pdf$', x).group(1)))
        newest_versions.append(newest_version)

    return newest_versions

# Function to remove duplicated files
def remove_duplicates(folder):
    newest_versions = find_newest_versions(folder)

    for filename in os.listdir(folder):
        if filename not in newest_versions:
            file_path = os.path.join(folder, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)

remove_duplicates(dataset_path)

In [11]:
# Split PDF files into different folders to use as training, evaluation and test data
training_data_path = "F:\Datasets/training/"
evaluation_data_path = "F:\Datasets/evaluation/"
test_data_path = "F:\Datasets/test/"

# Define the number of PDF files we want to use for each set of data
training_data_limit = 98
evaluation_data_limit = 21
test_data_limit = 21

# Define the output file name
training_data_output_file = "dataset"
evaluation_data_output_file = "dataset_eval"
test_data_output_file = "dataset_test"

# Assign the variables to the corresponding dataset.
dataset_path = test_data_path
dataset_limit = test_data_limit
output_file = test_data_output_file

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:4: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:4: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Kotorin\AppData\Local\Temp\ipykernel_19884\3393441614.py:2: SyntaxWarning: invalid escape sequence '\D'
  training_data_path = "F:\Datasets/training/"
C:\Users\Kotorin\AppData\Local\Temp\ipykernel_19884\3393441614.py:3: SyntaxWarning: invalid escape sequence '\D'
  evaluation_data_path = "F:\Datasets/evaluation/"
C:\Users\Kotorin\AppData\Local\Temp\ipykernel_19884\3393441614.py:4: SyntaxWarning: invalid escape sequence '\D'
  test_data_path = "F:\Datasets/test/"


Step 1: Breakdown PDF content by sections and subsections.

In [5]:
def process_one_pdf(file_path):
    doc, total_text, _ = processing_pdf.open_file(file_path)
    table_of_content = doc.get_toc()

    if len(table_of_content) > 0:
        print("Auto generated table of content:")
        display(table_of_content)
        # separate content into sections
        _, json_dict = processing_pdf.separate_content(total_text, table_of_content)
        return json_dict
    # some papers have not table of content
    #if len(table_of_content) == 0:
    #    print("The paper has not table of content. Need to use regular expression to map table of content.")
    #    table_of_content = processing_pdf.auto_find_toc(doc)
    #    display(table_of_content)
    
    return {}

In [13]:
file_count = 0
all_json_dicts = []
# Loop through files in the folder
for file_name in os.listdir(dataset_path):
    file_path = os.path.join(dataset_path, file_name)
    # Check if it's a file and if it has a ".pdf" extension
    if os.path.isfile(file_path) and file_name.endswith('.pdf'):
        # Process the PDF file
        print("Processing PDF file:", file_name)
        json_dict = process_one_pdf(file_path)
        if len(json_dict) > 0:
            all_json_dicts.extend(list(json_dict.values()))
            file_count += 1
            print("Done with PDF file:", file_name)
            print("# of sections:", len(json_dict.values()))
            print("Total # of sections:", len(all_json_dicts), "Total # of files:", file_count)
            print(60*"=")
        
        # Terminate when reaching dataset limit
        if file_count >= dataset_limit:
            break

json_list = json.dumps(all_json_dicts)
output_file_path = f"{output_path}/{output_file}.json"
with open(output_file_path, "w") as jsonfile: 
    jsonfile.write(json_list)

Processing PDF file: 1901.00141v1.pdf
F:\Datasets/test/1901.00141v1.pdf
Processing PDF file: 1901.00142v2.pdf
F:\Datasets/test/1901.00142v2.pdf
Processing PDF file: 1901.00143v1.pdf
F:\Datasets/test/1901.00143v1.pdf
Processing PDF file: 1901.00144v2.pdf
F:\Datasets/test/1901.00144v2.pdf
Auto generated table of content:


[[1, '1. Introduction', 1],
 [1, '2. Topology of the tempered dual', 3],
 [1,
  '3. Topology of the motion group dual and remarks on the Mackey-Higson bijection',
  6],
 [1, '4. Continuity of the Mackey-Higson bijection', 8],
 [1, 'References', 11]]

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00144v2.pdf
# of sections: 6
Total # of sections: 6 Total # of files: 1
Processing PDF file: 1901.00145v3.pdf
F:\Datasets/test/1901.00145v3.pdf
Auto generated table of content:


[[1, '1. Introduction', 1],
 [1, '2. Preliminaries', 8],
 [1, '3. The Thom Isomorphism', 14],
 [1, '4. Poincaré Duality', 16],
 [1, '5. Doubling', 22],
 [1, '6. Finite Coverings', 24],
 [1, '7. Fibrations', 25],
 [1, '8. Historical Remarks', 27],
 [1, 'Appendix A. Skew-Commutativity of Cup Products', 28],
 [1, 'Appendix B. The Künneth Theorems', 30],
 [1, 'Appendix C. Proof of Theorem 3.6', 35],
 [1, 'References', 38]]

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00145v3.pdf
# of sections: 13
Total # of sections: 19 Total # of files: 2
Processing PDF file: 1901.00146v1.pdf
F:\Datasets/test/1901.00146v1.pdf
Auto generated table of content:


...

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00168v1.pdf
# of sections: 2
Total # of sections: 115 Total # of files: 19
Processing PDF file: 1901.00169v1.pdf
F:\Datasets/test/1901.00169v1.pdf
Processing PDF file: 1901.00170v1.pdf
F:\Datasets/test/1901.00170v1.pdf
Auto generated table of content:


[[1, '1. Introduction and main result', 1],
 [1, '2. Technical lemmas', 2],
 [1, '3. Proof of Theorem ??', 3],
 [1, '4. Extension to integer points close to smooth curves', 5],
 [1, 'References', 5]]

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00170v1.pdf
# of sections: 6
Total # of sections: 121 Total # of files: 20
Processing PDF file: 1901.00171v2.pdf
F:\Datasets/test/1901.00171v2.pdf
Auto generated table of content:


[[1, '1 Introduction', 1],
 [1, '2 Related Work', 2],
 [1, '3 Measurement and Observation', 2],
 [1, '4 DCA: Disparity-preserved Deep Cross-platform Association', 3],
 [2, '4.1 Problem Formulation', 3],
 [2, '4.2 Disparity-preserved Deep Cross-platform Association', 3],
 [1, '5 Cross-platform Video Recommendation', 4],
 [2, '5.1 Problem Formulation', 4],
 [2, '5.2 DCA-based Cross-platform Video Recommendation', 4],
 [1, '6 Experiments', 5],
 [2, '6.1 Dataset', 5],
 [2, '6.2 Experimental Settings', 5],
 [3, 'Linear Regression-based Association (LR)', 5],
 [3, 'Latent Attribute-based Association (LA)', 5],
 [3, 'MLP-based Nonlinear Mapping (MLP)', 5],
 [2, '6.3 Evaluation of Cross-platform Association', 5],
 [2, '6.4 Evaluation of Cross-platform Video Recommendation', 6],
 [1, '7 Conclusion', 6]]

starting looking for all the sections according to the provided section title info...
Done with PDF file: 1901.00171v2.pdf
# of sections: 8
Total # of sections: 129 Total # of files: 21


Step 1.2: Generated data validation

In [14]:
with open(f"{output_path}/{output_file}.json",  encoding='utf-8') as f:
    data = json.load(f)

total_objects = len(data)
empty_text_count = 0
sub_sections_exist_count = 0

# Iterate over each section
for obj in data:
    if obj['Text'] == '' and not 'reference' in obj['Section'].lower():
        empty_text_count += 1
    if len(obj['Subsections']) < 1:
        sub_sections_exist_count += 1

# Print the results
print("Total number of sections:", total_objects)
print("Number of sections with empty Text:", empty_text_count)
print("Number of sections with sub sections:", sub_sections_exist_count)

Total number of sections: 129
Number of sections with empty Text: 23
Number of sections with sub sections: 114


Step 2: Generate GPT summary for each section and subsection as ground truth.

In [ ]:
from gpt_summary import get_summaries

# output_file = training_data_output_file
with open(f"{output_path}/{output_file}.json", encoding='utf-8') as f:
    data = json.load(f)
get_summaries(data)

output_file_path_ground_truth = f"{output_path}/{output_file}_ground_truth.json"
with open(output_file_path_ground_truth, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)